<a href="https://colab.research.google.com/github/fabricecordelieres/IJ-Toolset_Root-Photoactivation-Analysis/blob/main/Python_Script/Colab_Root_Photoactivation_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Batch Data Pulling Notebook
---
*This notebook takes as an input a folder containing one subfolder per condition, each containing individual csv files. Each csv file contains fluorescence information about cell 0 (Drompa activated cell) and the two receiving/bording cells (cell -1 and cell 1). This notebook will organise the data into one excel workbook per condition, each acquisition being copied to a single spreadsheet. Data will be normalised to 100% for cell 0 immediately after activation. A summary spreadsheet will be generated containing mean, SD and n for each condition. For both individual acqusitions and summarized data, a chart will be generated*



## Data organization

**Input folder:**

---
  - *Sub-folder 1: condition 1*
    - Raw data, acquisition 1, as a csv file 
    - Raw data, acquisition 2, as a csv file
    - ...
    - Raw data, acquisition m, as a csv file
---
  - *Sub-folder 2: condition 2*
    - Raw data, acquisition 1, as a csv file 
    - Raw data, acquisition 2, as a csv file
    - ...
    - Raw data, acquisition m, as a csv file
---
  - *...*
---
  - *Sub-folder n: condition n*
    - Raw data, acquisition 1, as a csv file 
    - Raw data, acquisition 2, as a csv file
    - ...
    - Raw data, acquisition m, as a csv file
---


## **How to run this script**

1.   Have all you data ready on the Google Drive: one folder with one sub-folder per condition, all the raw data files in each sub-folder
2.   Run each steps: press the play button, on the left side of the relevent cells
  1. Step 1 only is interactive: it will require that acces is given by the Google Drive to the script. Simply follow the procedure explained at Step 1.1
  2. Step 1.2 is also interactive: give the path to the folder containing all files, then run the cell.
3. At the end of the script, for each file, a file_with-NN is created in the same folder as the source data.


#Step 1: Prepare the environment for execution

## **_Step 1.1: Install librairies_**

In [1]:
#@markdown _Simply execute the following cell_

! pip install XlsxWriter

#from google.colab import drive
#import math
#import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 4.2 MB/s 


## **_Step 1.2: Declare some helping functions_**
_Defines procedures to load the data csv files and create xlsx formatted files_

## **_Step 1.2.1: Defines how to process csv files_**

In [2]:
#@markdown _Simply execute the following cell_

#-----------------LOAD AND MANIPULATE CSV FILES-----------------
import pandas as pd

class read_csv():
  """Reads a single csv file, from an input path, 
  and performs the required data formatting

  Keyword argument:
    path -- path to the data file"""

  #Class attibute
  data=pd.DataFrame()

  def __init__(self, path=''):
    self.data=pd.read_csv(path).iloc[0::2].reset_index(drop=True) # lines 1, 3, 4...
    # self.data=pd.read_csv(path).iloc[1::2] lines 2, 4, 6...

  def get_data(self):
    """Returns the relevent data, as a DataFrame"""
    return self.data

## **_Step 1.2.2: Defines how to read params.txt file_**

In [3]:
#@markdown _Simply execute the following cell_

#-----------------LOAD AND MANIPULATE PARAMS.TXT FILES-----------------
class read_param():
  """Reads the parameters file, from an input path, 

  Keyword argument:
    path -- path to the parameters file"""

  #Stores parameters as key values
  params={}
  
  #List of subfolders
  paths=''

  #Number of bording layers
  n_bording_layers=0

  def __init__(self, path=''):
    file = open(path, 'r')
    lines = file.readlines()

    self.params = dict(line.rstrip().split('=',1) for line in lines)

    self.paths=self.params['paths'].split(',')
    self.n_bording_layers=int(self.params['nBordingCells'])

## **_Step 1.2.3: Defines how to compile csv files to a xlsx file_**

In [8]:
#@markdown _Simply execute the following cell_

#-----------------CREATE OUTPUT XLS FILE-----------------
import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell
import os
import re

class create_XLSX():
  """Creates a XLSX file, in an output folder, 
  adds one sheet per input file,
  adds the formula and charts,
  adds a compiled data and a summary sheet

  Keyword argument:
    path -- path to the data file"""

  
  #Class attibute
  in_path=''
  dataset=''
  files_list=''
  out_path=''
  workbook_path=''
  workbook = ''
  title_format=''
  headers_format=''
  cells_timesteps_format=''
  cells_format=''

  #Hard-coded parameters
  timestamps=''
  rois_labels=''
  data_labels=''

  n_bording_layers=0 #Number of bording layers

  summary='' #The summary worksheet

  #-----------------------------------------------------------------------------  
  #-----------------------------------------------------------------------------
  def __init__(self, in_path='', out_path='', n_bording_layers=0):
    #Initializes headers
    self.timestamps=['Before activation', 'Activation', '2 sec activation', 15, 30, 45, 60, 75, 90, 105, 120]
    self.rois_labels=['Background', 'Activated Cell']
    self.data_labels=['Raw data', 'Background subtracted', 'Normalized to 1', 'Normalized to 100%']


    #Prepares path and names
    self.in_path=in_path
    tmp=in_path.split('/')
    self.dataset=tmp[len(tmp)-2] # the path is supposed to end with /: last element of the table is therefore empty
    self.out_path=out_path
    self.n_bording_layers=n_bording_layers

    #Prepares the labels
    for i in range(1, n_bording_layers+1):
      self.rois_labels.append('Border Cell '+str(i))
      self.rois_labels.append('Border Cell '+str(i)+"'")

    #Checks for previous XLSX output and removes it if it exists
    self.workbook_path=self.out_path+self.dataset+'.xlsx'
    if os.path.exists(self.workbook_path):
      os.remove(self.workbook_path)

    #Get the list of all files to analyze
    self.files_list=[f.replace('.csv', '') for f in os.listdir(in_path) if (os.path.isfile(os.path.join(in_path, f)) and f.endswith('.csv'))]
    self.files_list.sort()

    #Creates the XLSX output
    self.workbook = xlsxwriter.Workbook(self.workbook_path)

    # Create a format to use for title
    self.title_format = self.workbook.add_format({
      'bold': 1,
      'border': 1,
      'align': 'center',
      'valign': 'vcenter',
      'text_wrap': 1,
      'shrink': 1})
  
    # Create a format to use for headers
    self.headers_format = self.workbook.add_format({
      'bold': 1,
      'border': 1,
      'align': 'center',
      'valign': 'vcenter',
      'text_wrap': 1,
      'shrink': 1})
    
    # Create a format to use for cells
    self.cells_timesteps_format = self.workbook.add_format({
      'bold': 1,
      'border': 1,
      'align': 'center',
      'valign': 'vcenter',
      'text_wrap': 1,
      'shrink': 1})
    
    # Create a format to use for cells
    self.cells_format = self.workbook.add_format({
      'border': 1,
      'align': 'center',
      'valign': 'vcenter',
      'text_wrap': 1,
      'shrink': 1})

  #-----------------------------------------------------------------------------  
  #-----------------------------------------------------------------------------
  def create_all_worksheets(self):
    """Creates and feeds all the spreadsheets based on the content of the input folder"""

    print('******* Processing dataset '+self.dataset+' *******')
    self.summary = self.workbook.add_worksheet('Summary')
    print('Added Summary worksheet')
    for f in self.files_list:
      self.create_and_feed_single_worksheet(f)
    self.feed_summary_worksheet()
    self.workbook.close()

    print('******* Done *******')

  #-----------------------------------------------------------------------------  
  #-----------------------------------------------------------------------------
  def create_and_feed_single_worksheet(self, name):
    """Creates a single spreasheet from a single csv file

    Keyword argument:
    name -- name of the input csv file"""

    worksheet = self.workbook.add_worksheet(name)
    print('Added', name, 'worksheet')

    #Reads data
    input_csv=read_csv(self.in_path+name+".csv")
    data=input_csv.get_data()

    # Define offsets to put data and hard coded data
    offset_row=4 #Depends on the number of lines for the headers
    block_size=2*(2+self.n_bording_layers*2) #Headers' block size= (Bkgd+Activated+ 2*cells/layer)*2columns
    
    # --------Handles the raw data--------
    #Adds general headers
    worksheet.merge_range(0, 0, 0, block_size*len(self.data_labels), name, self.title_format)
    worksheet.merge_range('A2:A4', 'Time', self.headers_format)

    #Adds data header
    index_data=0

    for label in self.data_labels:
      worksheet.merge_range(1, block_size*index_data+1, 1, block_size*index_data+block_size, label, self.headers_format)
      
      index_roi=0
      for roi in self.rois_labels:
        #Adds cell/data headers
        worksheet.merge_range(2, 2*index_roi+1+block_size*index_data, 2, 2*index_roi+2+block_size*index_data, roi, self.headers_format)
        worksheet.write(3, 2*index_roi+1+block_size*index_data, 'Area', self.headers_format)
        worksheet.write(3, 2*index_roi+2+block_size*index_data, data['Area('+self.rois_labels[index_roi]+')'][0], self.headers_format)
        index_roi=index_roi+1

      index_data=index_data+1

    
    #Adds timestamp
    index=0
    for row in self.timestamps:
      worksheet.write(index+offset_row, 0, row, self.cells_timesteps_format)
      index=index+1

    #Adds raw data
    index_roi=0
    nCol_per_dataType=2*len(self.rois_labels)
    area_background='$C$4'  
    for roi in self.rois_labels:
      #Locate the area cells
      area_curr_cell='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(3, 2*index_roi+2))[0:2])

      for row in range(len(data['RawIntDen('+self.rois_labels[index_roi]+')'])):
        
        worksheet.merge_range(row+offset_row, 2*index_roi+1, row+offset_row, 2*index_roi+2, data['RawIntDen('+self.rois_labels[index_roi]+')'][row], self.cells_format)
        
        #Before activation subtracted
        raw_background=xl_rowcol_to_cell(row+offset_row, 1)
        raw_current_cell=xl_rowcol_to_cell(row+offset_row, 2*index_roi+1)

        worksheet.merge_range(row+offset_row, 2*index_roi+1+nCol_per_dataType, row+offset_row, 2*index_roi+2+nCol_per_dataType, '='+raw_current_cell+'-'+raw_background+'*'+area_curr_cell+'/'+area_background, self.cells_format)
        
        #Adds formulas for computations
        #Normalized to 1 and %
        sum_cells='+'.join([xl_rowcol_to_cell(row+offset_row, 2*cell+1+nCol_per_dataType) for cell in range(1, 2*self.n_bording_layers+2)])
        
        curr_cell=xl_rowcol_to_cell(row+offset_row, 2*index_roi+1+nCol_per_dataType)
        formula='='+curr_cell+'/('+sum_cells+')'

        worksheet.merge_range(row+offset_row, 2*index_roi+1+2*nCol_per_dataType, row+offset_row, 2*index_roi+2+2*nCol_per_dataType, formula, self.cells_format)
        worksheet.merge_range(row+offset_row, 2*index_roi+1+3*nCol_per_dataType, row+offset_row, 2*index_roi+2+3*nCol_per_dataType, formula.replace('=', '=100*'), self.cells_format)
      index_roi=index_roi+1

    #Formats the data
    #Column width
    worksheet.set_column(0, 0, 14.5)
    worksheet.set_column(1, 1, 9)
    worksheet.set_column(1, 2, 9)

    #Adds graphs
    #graph=self.workbook.add_chart({'type': 'scatter', 'subtype': 'smooth_with_markers'})
    graph=self.workbook.add_chart({'type': 'line'})


    # Configure the data series.
    for roi in range(len(self.rois_labels)):
      #Defines cells locations
      n_lines=len(data['RawIntDen('+self.rois_labels[roi]+')'])-1+offset_row
      col_data=2*roi+1+3*nCol_per_dataType

      name_cell='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(2, col_data))[0:2])
      categories_cell_start='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(offset_row+1, 0))[0:2])
      categories_cell_end='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(n_lines, 0))[0:2])
      values_cells_start='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(offset_row+1, col_data))[0:2])
      values_cells_end='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(n_lines, col_data))[0:2])
      
      #Adds series to the graph
      graph.add_series({
          'name':       '='+name+'!'+name_cell,
          'categories': '='+name+'!'+categories_cell_start+':'+categories_cell_end,
          'values':     '='+name+'!'+values_cells_start+':'+values_cells_end,
          #'data_labels': {'series_name': True},
      })

    #Graph formatting
    graph.set_title({'name': name})
    graph.set_x_axis({
        'name' : 'Timepoint',
        'text_axis': True,
        'num_font':  {'rotation': -45},
    })
    graph.set_y_axis({
        'name' : 'Normalized fluorescence (%)',
    })
    graph.set_size({
        'width': 1024,
        'height': 512,
    })

    #Inserts the graph
    insert_graph_cell=xl_rowcol_to_cell(n_lines+2, 0)
    worksheet.insert_chart(insert_graph_cell, graph)

    #Inserts image
    insert_img_cell=xl_rowcol_to_cell(n_lines+2, 16)
    worksheet.insert_image(insert_img_cell, self.in_path+name+".jpg")

  #-----------------------------------------------------------------------------  
  #-----------------------------------------------------------------------------
  def feed_summary_worksheet(self):
    """Feeds a summary spreasheet"""

    n_rois=len(self.rois_labels) 
    n_files=len(self.files_list)
    n_timepoints=len(self.timestamps)
    n_value_types=len(self.data_labels)
    n_col_per_data=2
    offset_row=1;

    #Adds headers
    self.summary.merge_range(0, 0, 0, (n_files+3)*(n_rois-1), 'Data summary for '+self.dataset, self.title_format)
    self.summary.merge_range('A2:A3', 'Time', self.headers_format)

    index=1
    for roi in range(1, n_rois): #Starts after background ROI
      self.summary.merge_range(offset_row, (roi-1)*(n_files+3)+1, offset_row, roi*(n_files+3), self.rois_labels[roi], self.headers_format)
      for dataset in self.files_list:
        self.summary.write(offset_row+1, index, dataset, self.headers_format)
        index=index+1
      self.summary.write(offset_row+1, index, 'Mean', self.headers_format)
      self.summary.write(offset_row+1, index+1, 'SD', self.headers_format)
      self.summary.write(offset_row+1, index+2, 'n', self.headers_format)
      index=index+3

    #Adds timestamp
    offset_row=offset_row+2

    index=0
    for row in self.timestamps:
      self.summary.write(index+offset_row, 0, row, self.cells_timesteps_format)
      index=index+1
    
    #Adds references to data and computes stats
    for timepoint in range(0, n_timepoints):
      index=1
      for roi in range(1, n_rois): #Starts after background ROI
        cell_to_ref=xl_rowcol_to_cell(offset_row+timepoint+1, 1+(n_col_per_data*n_rois)*(n_value_types-1)+2*roi)
        data_cells=[]
        for dataset in self.files_list:
          self.summary.write(offset_row+timepoint, index, '='+dataset+'!'+cell_to_ref, self.cells_format)
          data_cells.append(xl_rowcol_to_cell(offset_row+timepoint, index))
          index=index+1
        self.summary.write(offset_row+timepoint, index, '=AVERAGE('+data_cells[0]+':'+data_cells[len(data_cells)-1]+')', self.headers_format)
        self.summary.write(offset_row+timepoint, index+1, '=STDEV('+data_cells[0]+':'+data_cells[len(data_cells)-1]+')', self.headers_format)
        self.summary.write(offset_row+timepoint, index+2, '=COUNT('+data_cells[0]+':'+data_cells[len(data_cells)-1]+')', self.headers_format)
        index=index+3

    #Formats the data
    #Column width
    self.summary.set_column(0, 0, 14.5)
    self.summary.set_column(1, 1, 9)
    self.summary.set_column(1, 2, 9)

    #Adds graphs
    graph=self.workbook.add_chart({'type': 'line'})

    n_lines=len(self.timestamps)-1
    categories_cell_start='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(3+1, 0))[0:2])
    categories_cell_end='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(3+n_lines, 0))[0:2])
      
    for roi in range(1, n_rois): #Starts after background ROI
      #Defines cells locations
      name_cell='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(1, 1+(n_files+3)*(roi-1)))[0:2])

      values_cells_start='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(3+1, 1+(n_files+3)*(roi-1)+n_files))[0:2])
      values_cells_end='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(3+n_lines, 1+(n_files+3)*(roi-1)+n_files))[0:2])

      sd_cells_start='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(3+1, 1+(n_files+3)*(roi-1)+n_files+1))[0:2])
      sd_cells_end='$'+'$'.join(re.split('(\d+)', xl_rowcol_to_cell(3+n_lines, 1+(n_files+3)*(roi-1)+n_files+1))[0:2])

      #Adds series to the graph
      graph.add_series({
          'name':       '=Summary!'+name_cell,
          'categories': '=Summary!'+categories_cell_start+':'+categories_cell_end,
          'values':     '=Summary!'+values_cells_start+':'+values_cells_end,
          'y_error_bars': {
            'type':         'custom',
            'plus_values':  '=Summary!'+sd_cells_start+':'+sd_cells_end,
            'minus_values': '=Summary!'+sd_cells_start+':'+sd_cells_end,
          },
          #'data_labels': {'series_name': True},
      })

    #Graph formatting
    graph.set_title({'name': 'Data summary for '+self.dataset})
    graph.set_x_axis({
        'name' : 'Timepoint',
        'text_axis': True,
        'num_font':  {'rotation': -45},
    })
    graph.set_y_axis({
        'name' : 'Normalized fluorescence (%)',
    })
    graph.set_size({
        'width': 1024,
        'height': 512,
    })

    #Inserts the graph
    insert_graph_cell=xl_rowcol_to_cell(n_lines+5, 0)
    self.summary.insert_chart(insert_graph_cell, graph)

## **_Step 1.2.4: Defines how to upload the data, launch analysis and download the results, while keeping the swapping space clean_**

In [5]:
#@markdown _Simply execute the following cell_

from google.colab import files
from zipfile import ZipFile
import shutil
import os
#-----------------UPLOAD DATA, PERFORM ANALYSIS AND DOWNLOAD THE RESULTS-----------------
class compile_data():
  """
  1-Uploads a zip file
  2-Unzips it
  3-Performs the analysis on all folders
  4-Zips the results
  5-Downloads the output zip file
  """

  #Class attibute
  uploaded=''
  in_dir=''
  in_file=''
  out_file=''

  def __init__(self):
    '''Uploads the input zip file and keeps track of it as a class attribute'''
    #File chooser
    self.uploaded = files.upload()

  def generate_output(self):
    '''Unzips the uploaded file, performs analysis, creates the output zip file and downlaods it'''
    #Unzip file(s), analyzes/downloads/clean up
    for filename in self.uploaded.keys():
      #Create a folder named after the file
      zip_extract_path='/content/'+filename.replace('.zip', '')
      os.mkdir(zip_extract_path)

      #Extract the file
      zip_in=ZipFile(filename, 'r')
      zip_in.extractall(zip_extract_path)
      zip_in.close()

      #Analyze
      self.in_file='/content/'+filename
      self.in_dir=self.in_file.replace('.zip', '/')
      self.out_file='/content/_dataAnalysed.zip'
      params=read_param(self.in_dir+'params.txt')

      out_dir=self.in_dir+'output/'
      if not os.path.exists(out_dir):
        os.mkdir(out_dir)

      for dataset in params.paths:
        in_path=self.in_dir+dataset
        curr_xls=create_XLSX(in_path, out_dir, params.n_bording_layers)
        curr_xls.create_all_worksheets()

      zip_out = ZipFile(self.out_file, "w")
      xlsx_files=[f for f in os.listdir(out_dir) if (os.path.isfile(os.path.join(out_dir, f)) and f.endswith('.xlsx'))]
      
      for f in xlsx_files:
        zip_out.write(os.path.join(out_dir, f), f)
      zip_out.close()

      files.download(self.out_file)
      self.clean_up_ze_mess()

  def clean_up_ze_mess(self):
    """Removes the input zip file and the extracted folders"""
    if(os.path.exists('/content/__MACOSX')):
      shutil.rmtree('/content/__MACOSX')
    shutil.rmtree(self.in_dir)
    os.remove(self.in_file)

  def delete_output_file(self):
    '''Deletes the output zip file: DO NOT CALL IT BEFORE DOWNLOAD HAS ENDED'''
    os.remove(self.out_file)

#Step 2: Performs the analysis: upload data, analyze, download results


## **_Step 2.1: Upload data, analyze and download results_**
_Processes the data_
_Simply execute the following cell_

In [9]:
where_the_magic_happens=compile_data()
where_the_magic_happens.generate_output()

Saving _dataToUpload.zip to _dataToUpload.zip
******* Processing dataset dronpa DMSO 24h copie 2 *******
Added Summary worksheet
Added FRAP worksheet
Added FRAP_001 worksheet
Added FRAP_002 worksheet
Added FRAP_003 worksheet
Added FRAP_004 worksheet
Added FRAP_005 worksheet
Added FRAP_006 worksheet
Added FRAP_007 worksheet
******* Done *******
******* Processing dataset dronpa DMSO 24h copie 3 *******
Added Summary worksheet
Added FRAP worksheet
Added FRAP_001 worksheet
Added FRAP_002 worksheet
Added FRAP_003 worksheet
Added FRAP_004 worksheet
Added FRAP_005 worksheet
Added FRAP_006 worksheet
Added FRAP_007 worksheet
******* Done *******
******* Processing dataset dronpa DMSO 24h copie *******
Added Summary worksheet
Added FRAP worksheet
Added FRAP_001 worksheet
Added FRAP_002 worksheet
Added FRAP_003 worksheet
Added FRAP_004 worksheet
Added FRAP_005 worksheet
Added FRAP_006 worksheet
Added FRAP_007 worksheet
******* Done *******
******* Processing dataset dronpa DMSO 24h *******
Added

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **_Step 2.2: Clean up temporary data_**
_Cleans up the workspace: **to be executed once the results have been generated and downloaded**_

In [7]:
where_the_magic_happens.delete_output_file()